In [1]:
%run './functions.ipynb'

In [2]:
meta_df = pd.read_excel(patietns_meta_data)
intensity_meta = load_intensity_meta_data(Path(direcroy_path)/Path(PREPROCESSED_FP_INTENSITY),'Gene names')
RNA_failed_patients = meta_df['Sample name'][meta_df['RNA QC issue'] == 'x'].tolist()

reading /media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/2023.06.22_AhS_PAPER_COHORT/preprocessed_fp.csv
Intensity meta_data loaded


# Correlation with  all Transcript FPKM


In [ ]:
#  full proteome_z_scores
Z_SCORES_df_proteome = pd.read_csv(Path(direcroy_path)/Path(Z_SCORES_PATH),sep='\t')
Z_SCORES_df_proteome = Z_SCORES_df_proteome.set_index('Gene names')
Z_SCORES_df_proteome = Z_SCORES_df_proteome.filter(regex=REGEX)
# there is no protein grouping at transcriptome level we unnest the protein groups also for protein data
Z_SCORES_df_proteome = unnest_proteingroups(Z_SCORES_df_proteome)
Z_SCORES_df_proteome.columns = Z_SCORES_df_proteome.columns.str.replace('zscore_','')

# transcriptome_z_scores
transcripts_z_scores = pd.read_csv(transcripts_path)
transcripts_z_scores = transcripts_z_scores.set_index('genes')
transcripts_z_scores =  transcripts_z_scores.filter(regex=Z_SCORE_REGEX)
transcripts_z_scores.columns = transcripts_z_scores.columns.str.replace(' Z-score','')

protein_list = intersection(transcripts_z_scores.index.tolist(),Z_SCORES_df_proteome.index.tolist())
intersecting_patients = intersection(transcripts_z_scores.columns.tolist(),Z_SCORES_df_proteome.columns.tolist())

intersecting_patients = [x for x in intersecting_patients if x not in  RNA_failed_patients]
Z_SCORES_df_proteome = Z_SCORES_df_proteome[intersecting_patients]
transcripts_z_scores = transcripts_z_scores[intersecting_patients]
correlation_df = pd.DataFrame(protein_list,columns=['protein'])
correlation_df['correlation'] = None
correlation_df['num_patients'] = None
correlation_df['median_peptides'] = None

for i in range(len(correlation_df)):
    protein_correlaiton, num_patientes = get_correlation(correlation_df['protein'][i],Z_SCORES_df_proteome,transcripts_z_scores)
    correlation_df['correlation'][i] = protein_correlaiton
    correlation_df['num_patients'][i] = num_patientes
    try:
        correlation_df['median_peptides'][i] = get_pep_number_from_protein_name(intensity_meta,correlation_df['protein'][i]).median()
    except:
        print('no median of the peptides was calculated for this protein')
correlation_df = correlation_df.set_index('protein')

basket_annotation_df = pd.read_excel(basket_annotation_path)
correlation_df['basket_annotation'] = ''
for i in range(len(correlation_df)):
    correlation_df['basket_annotation'][i] = ';'.join(basket_annotation_df['BASKET'][basket_annotation_df["GENE NAME"].isin([correlation_df.index[i]])].unique().tolist())

correlation_df = correlation_df.dropna(subset='correlation')
correlation_df.to_excel('~/Desktop/correlation_all_proteins_RNA_failed_removed.xlsx')


In [ ]:
get_corelation_scatter_plot_per_prtein('NF1',transcripts_z_scores,Z_SCORES_df_proteome,plot_saving_dir)


# Correlation df with the Lysates

In [ ]:
# after addin the ne
meta_data = pd.read_excel(patietns_meta_data)
lysate_samples = meta_data['Sample name'][meta_data['Input Material'] == 'Lysate']
#  full proteome_z_scores
Z_SCORES_df_proteome = pd.read_csv(Path(direcroy_path)/Path(Z_SCORES_PATH),sep='\t')
Z_SCORES_df_proteome = Z_SCORES_df_proteome.set_index('Gene names')
Z_SCORES_df_proteome = Z_SCORES_df_proteome.filter(regex=REGEX)
# there is no protein grouping at transcriptome level we unnest the protein groups also for protein data
Z_SCORES_df_proteome = unnest_proteingroups(Z_SCORES_df_proteome)
Z_SCORES_df_proteome.columns = Z_SCORES_df_proteome.columns.str.replace('zscore_','')

# transcriptome_z_scores
transcripts_z_scores = pd.read_csv(transcripts_path)
transcripts_z_scores = transcripts_z_scores.set_index('genes')
transcripts_z_scores =  transcripts_z_scores.filter(regex=Z_SCORE_REGEX)
transcripts_z_scores.columns = transcripts_z_scores.columns.str.replace(' Z-score','')


samples_to_consider = intersection(transcripts_z_scores.columns,lysate_samples)
samples_to_consider = intersection(samples_to_consider,Z_SCORES_df_proteome.columns)
samples_to_consider = [x for x in samples_to_consider if x not in  RNA_failed_patients]

transcripts_z_scores = transcripts_z_scores[samples_to_consider]
Z_SCORES_df_proteome = Z_SCORES_df_proteome[samples_to_consider]


protein_list = intersection(transcripts_z_scores.index.tolist(),Z_SCORES_df_proteome.index.tolist())
intersecting_patients = intersection(transcripts_z_scores.columns.tolist(),Z_SCORES_df_proteome.columns.tolist())
Z_SCORES_df_proteome = Z_SCORES_df_proteome[intersecting_patients]
transcripts_z_scores = transcripts_z_scores[intersecting_patients]
correlation_df_lysates = pd.DataFrame(protein_list,columns=['protein'])
correlation_df_lysates['correlation'] = None
correlation_df_lysates['num_patients'] = None
for i in range(len(correlation_df)):
    protein_correlaiton, num_patientes = get_correlation(correlation_df_lysates['protein'][i],Z_SCORES_df_proteome,transcripts_z_scores)
    correlation_df_lysates['correlation'][i] = protein_correlaiton
    correlation_df_lysates['num_patients'][i] = num_patientes


correlation_df_lysates = correlation_df_lysates.set_index('protein')
basket_annotation_df = pd.read_excel(basket_annotation_path)
correlation_df_lysates['basket_annotation'] = ''
for i in range(len(correlation_df_lysates)):
    correlation_df_lysates['basket_annotation'][i] = ';'.join(basket_annotation_df['BASKET'][basket_annotation_df["GENE NAME"].isin([correlation_df_lysates.index[i]])].unique().tolist())


correlation_df_lysates = correlation_df_lysates.dropna(subset='correlation')
correlation_df_lysates.to_excel('~/Desktop/correlation_df_with_lysates_RNA_failed_removed.xlsx')
#samples = pd.DataFrame(samples_to_consider)
#samples.columns = ['IDs']
correlation_df['protein'] = correlation_df.index
correlation_df['type'] = 'all'
correlation_df_lysates['protein'] = correlation_df_lysates.index
correlation_df_lysates['type'] = 'Lysate'

all_correlation_df = pd.concat([correlation_df_lysates,correlation_df])
all_correlation_df = all_correlation_df.dropna(subset='correlation')


# Histogram

In [ ]:
#import plotly.express as px
#fig = px.histogram(all_correlation_df, x="correlation",color='type',nbins=100,histnorm='probability density')
#fig.show()
# libraries & dataset
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(19.20,10.80))
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
#sns.set(style="darkgrid")
#fig, axs = plt.subplots(1, 2, figsize=(7, 7))
#sns.histplot(data=correlation_df_lysates, x="correlation",binwidth=0.1, kde=True, color="red", ax=axs[0],bins=1000)
#sns.histplot(data=correlation_df, x="correlation", kde=True, color="blue", ax=axs[1],bins=1000)
correlation_df = correlation_df.dropna(subset='correlation')
correlation_df_lysates = correlation_df_lysates.dropna(subset='correlation')
sns.histplot(data=correlation_df_lysates, x="correlation",label="Lysates",  kde=True,color="red",bins=50) # only for the lysates
sns.histplot(data=correlation_df, x="correlation",label="All patients", kde=True, color="blue",bins=50)  # for all patients
#plt.axvline(2.8, 0,0.17)
plt.legend() 
#plt.show()
plt.savefig(f'{plot_saving_dir}/correlation_FPKM_protein_lysates_vs_all.svg',dpi=300)



# intensity dynamic range

In [ ]:
# Dynamic range
intensity_file = Path(direcroy_path)/Path(PREPROCESSED_FP_INTENSITY)
intensity_file_df = pd.read_csv(intensity_file)
intensity_file_df = intensity_file_df.set_index('Gene names')
intensity_file_df = unnest_proteingroups(intensity_file_df)
intensity_file_data = intensity_file_df.filter(regex=REGEX)
intensity_file_meta_df = intensity_file_df.filter(regex=REGEX_META)

intensity_file_meta_df = intensity_file_meta_df.applymap(str)
intensity_file_meta_df = intensity_file_meta_df.applymap(replace_statement)
intensity_file_meta_df = intensity_file_meta_df.applymap(lambda x:x.replace('nan','0'))
intensity_file_meta_df = intensity_file_meta_df.apply(pd.to_numeric)


occurence = intensity_file_data.count(axis=1)
mins = intensity_file_data.min(axis=1)
maxes = intensity_file_data.max(axis=1)
medians = intensity_file_data.median(axis=1)

min_num_pep = intensity_file_meta_df.min(axis=1)
max_num_pep = intensity_file_meta_df.max(axis =1 )
median_num_pep = intensity_file_meta_df.median(axis=1)

intensity_file_data['Occurence'] = occurence
intensity_file_data['min_intensity'] = mins
intensity_file_data['max_intensity'] = maxes
intensity_file_data['median_intensity'] = medians

intensity_file_meta_df['min_num_pep'] = min_num_pep
intensity_file_meta_df['max_num_pep'] = max_num_pep
intensity_file_meta_df['median_num_pep'] = median_num_pep


In [ ]:
# merging correlation between transcripts and proteome
final = pd.concat([intensity_file_meta_df[['min_num_pep','max_num_pep','median_num_pep']],
                   intensity_file_data[['min_intensity','max_intensity','median_intensity','Occurence']]],axis=1)
final_df = final.merge(correlation_df,right_index=True,left_index=True,how='left')


# adding basket annotation 

In [ ]:
basket_annotation_df = pd.read_excel(basket_annotation_path)
final_df['basket_annotation'] = ''
for i in range(len(final_df)):
    final_df['basket_annotation'][i] = ';'.join(basket_annotation_df['BASKET'][basket_annotation_df["GENE NAME"].isin([final_df.index[i]])].unique().tolist())


# TUPAC score dynamic range

In [ ]:
basket_scores_df = pd.read_csv(Path(direcroy_path)/BASKET_SCORES_PATH,sep='\t')
basket_scores_df = basket_scores_df.set_index('Sample').T
basket_scores_df['min_TUPAC_scores'] = basket_scores_df.min(axis=1)
basket_scores_df['max_TUPAC_scores'] = basket_scores_df.max(axis=1)
basket_scores_df['median_TUPAC_scores'] = basket_scores_df.median(axis=1)
TUPAC_scores_statistics = basket_scores_df[['min_TUPAC_scores','max_TUPAC_scores','median_TUPAC_scores']]

# saving the data

In [ ]:
with pd.ExcelWriter(Path(direcroy_path)/'dynamic_ranges_Correlation.xlsx') as writer:  
    final_df.to_excel(writer, sheet_name='FP')
    TUPAC_scores_statistics.to_excel(writer, sheet_name='TUPAC_SCORES')